In [1]:
from google.colab import drive 
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
cd /content/drive/'My Drive'/annd/nlp

/content/drive/My Drive/annd/nlp


In [3]:
import pandas as pd
import re
import os
import torch
import numpy as np

In [4]:
!pip install transformers

     |████████████████████████████████| 675kB 6.5MB/s 
     |████████████████████████████████| 890kB 19.4MB/s 
     |████████████████████████████████| 3.8MB 40.0MB/s 
     |████████████████████████████████| 1.1MB 36.8MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=39377850f79e863a21f5248df73489a94ffe6d4e041957b6f53091a57223b923
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [5]:
import numpy as np
from transformers import BertForSequenceClassification, BertTokenizer, AutoTokenizer
import torchtext
from torchtext import data
import torch
import re

In [6]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=4)
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
model = model.to(device)

In [7]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

In [8]:
Title = data.Field(sequential=True,tokenize=tokenizer.tokenize, lower=True, batch_first=True, init_token='[CLS]', eos_token='[SEP]',pad_token='[PAD]',unk_token='[UNK]')
Category = data.Field(sequential=False, batch_first=True, use_vocab=False)

train, valid, test = data.TabularDataset.splits(path = '.', train='train1.txt',
    validation='valid1.txt', test='test1.txt', format='tsv',
    fields=[('Title', Title), ('Category', Category)])

In [9]:
train_iter = torchtext.data.BucketIterator(dataset=train, batch_size=1, sort_key=lambda x: len(x.Title), shuffle = True)
valid_iter = torchtext.data.BucketIterator(dataset=valid, batch_size=1, sort_key=lambda x: len(x.Title), shuffle = True)
test_iter = torchtext.data.BucketIterator(dataset=test, batch_size=1,  sort_key=lambda x: len(x.Title), shuffle = True)

In [10]:
Title.build_vocab(train, min_freq=1)
Category.build_vocab(train)

In [11]:
opt = torch.optim.Adam(model.parameters(), lr=0.00001)

In [12]:
EPOCHS = 15

In [13]:
def accuracy(y_actual, y_pred):
    return np.sum(y_actual == y_pred).astype('int') / y_actual.shape[0]

In [14]:
for epoch in range(EPOCHS):
      avg_acc = 0 
      model.train()
      for i, batch in enumerate(train_iter):
        x, y = batch.Title, batch.Category
        loss, logit = model(input_ids=x.to(device), labels=y.to(device))
        opt.zero_grad()
        loss.backward()
        opt.step()
        print('\rEpoch: %d/%d, %f%% loss: %0.2f'% (epoch+1, EPOCHS, i/len(train_iter)*100, loss.item()), end='')
      for j, batch in enumerate(valid_iter):
        x, y = batch.Title, batch.Category
        loss, logit = model(input_ids=x.to(device), labels=y.to(device))
        _, y_pred_train = torch.max(logit, 1)
        acc = accuracy(y.cpu().numpy(), y_pred_train.cpu().detach().numpy().squeeze())
        avg_acc += acc
        print('\r%0.2f%% loss: %0.2f, accuracy %0.2f'% (j/len(valid_iter)*100, loss.item(), acc), end='')
      print('\nAverage accuracy: ', avg_acc / len(valid_iter))

99.93% loss: 0.16, accuracy 1.00
Average accuracy:  0.7256371814092953
99.93% loss: 0.16, accuracy 1.00
Average accuracy:  0.7263868065967016
99.93% loss: 0.03, accuracy 1.00
Average accuracy:  0.7931034482758621
99.93% loss: 0.03, accuracy 1.00
Average accuracy:  0.802848575712144
99.93% loss: 0.01, accuracy 1.00
Average accuracy:  0.8170914542728636
99.93% loss: 0.01, accuracy 1.00
Average accuracy:  0.8223388305847077
99.93% loss: 0.02, accuracy 1.00
Average accuracy:  0.8395802098950524
99.93% loss: 0.01, accuracy 1.00
Average accuracy:  0.8508245877061469
99.93% loss: 1.14, accuracy 0.00
Average accuracy:  0.848575712143928
Epoch: 10/15, 7.693028% loss: 0.00

KeyboardInterrupt: ignored

In [15]:
avg_acc = 0 
model.eval()
for i, batch in enumerate(test_iter):
        x, y = batch.Title, batch.Category
        loss, logit = model(input_ids=x.to(device), labels=y.to(device))
        _, y_pred_train = torch.max(logit, 1)
        acc = accuracy(y.cpu().numpy(), y_pred_train.cpu().detach().numpy().squeeze())
        avg_acc += acc
        print('\r%0.2f%% loss: %0.2f, accuracy %0.2f'% (i/len(test_iter)*100, loss.item(), acc), end='')
print('\nAverage accuracy: ', avg_acc / len(test_iter))

99.93% loss: 0.02, accuracy 1.00
Average accuracy:  0.8748125937031485
